In [1]:
import pandas as pd
#import pymysql
import os
import numpy as np
import re
import itertools
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import word_tokenize
import nltk
from joblib import Parallel, delayed
import pickle
from tqdm import tqdm

### Load final ABT Table

In [2]:
datapath = os.path.join(os.path.dirname(os.getcwd()), "data")
print(datapath)
if not os.path.exists(datapath):
    os.makedirs(datapath)

#df_master = pd.read_csv(os.path.join(datapath, "Final_ABT.csv"))

/home/ubuntu/matspec/data


In [10]:
df_IHS = pd.read_csv(os.path.join(datapath, "IhsPartDetails.csv"),
                           sep = "|", 
                           usecols = ['objectId', 'detail', 'content', "category", "description"]
                    )
#tmp = df_IHS.loc[df_IHS.category.str.contains("Rela") == True]

In [11]:
df_IHS.head()

objectId             detail              category content                               description
0  1000387133      Mfr Part Page    Active Part Master     NaN  Hyperlink to the manufacturer's web page
1  1000387133  Country of Origin    Active Part Master     NaN                         Country of Origin
2  1000387133           Features    Active Part Master     NaN            Important features of the part
3  1000387133            LC Risk  Auxiliary Part Class     Low                           Life Cycle Risk
4  1000387133           ENV Risk  Auxiliary Part Class     Low               Environmental/Material Risk

In [12]:
df_IHS.shape

(26716270, 5)

In [13]:
df_IHS.drop_duplicates(inplace=True)

In [14]:
df_IHS.shape

(26716097, 5)

In [19]:
df_IHS.describe(include = 'all')

objectId             detail  category  content        description
count   2.671610e+07           26716097  26716097  9671129           26716097
unique           NaN               1328        49   119103               1647
top              NaN  Temperature Grade    Diodes        1  Supply Chain Risk
freq             NaN             297334   9915429   505434             290293
mean    3.176769e+09                NaN       NaN      NaN                NaN
std     7.296071e+09                NaN       NaN      NaN                NaN
min     1.000077e+08                NaN       NaN      NaN                NaN
25%     1.350646e+09                NaN       NaN      NaN                NaN
50%     1.768085e+09                NaN       NaN      NaN                NaN
75%     2.106584e+09                NaN       NaN      NaN                NaN
max     1.450668e+11                NaN       NaN      NaN                NaN

In [18]:
df = df_IHS.dropna(subset = ["detail","category","content"])
df.shape

(9671129, 5)

In [22]:
df_value_overview = df.groupby(["category", "detail"]).size().reset_index(name='counts')
#df_grouped = df_IHS.groupby(["category", "detail"]).apply()

In [23]:
df_value_overview.to_csv(os.path.join(datapath, "overview_ihs.csv"), sep = "|", index = False)

In [55]:
value_dict["Supply Voltage-Min (Vsup) (V)"]

value     count
0     4.5  0.340074
1     2.7  0.149722
2       3  0.113358
3     1.7  0.039518
4     2.5  0.035993
5    4.75  0.033952
6       2  0.030056
7     1.2  0.025046
8     1.8  0.023006
9       1  0.020223
10    2.2  0.013544
11  1.283  0.012430
12    2.3  0.012430
13  2.495  0.009091
14      5  0.009091
15   1.65  0.008349
16      0  0.006865
17      8  0.006494
18  1.425  0.005751
19   1.14  0.005380
20      4  0.005009
21   10.8  0.005009
22   13.5  0.004638
23    3.5  0.004453
24   1.24  0.004267
..    ...       ...
71   4.25  0.000371
72   2.16  0.000371
73    1.3  0.000371
74  2.098  0.000371
75    2.9  0.000371
76    7.2  0.000371
77      6  0.000371
78    0.9  0.000371
79    9.5  0.000371
80    3.1  0.000371
81   0.55  0.000186
82    3.7  0.000186
83  14.25  0.000186
84   14.5  0.000186
85   2.64  0.000186
86    3.2  0.000186
87   12.5  0.000186
88    2.6  0.000186
89    7.5  0.000186
90   0.85  0.000186
91    6.9  0.000186
92  1.235  0.000186
93   0.75  0.000186
94    4.7  0.000186
95   2.35  0.000186

[96 rows x 2 columns]

In [14]:
attributes = df_IHS.detail[all(df_IHS.detail.value_counts() > 10 == True)]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [4]:
df_altMatIHS = pd.read_csv(os.path.join(datapath, "IhsAltMat.csv"), sep = "|")
df_altMatIHS.head()

id    objectId altType manufacturerPartNumber  alternativeObjectId  \
0   6  1001407707     FFF             CF14JA120R           1004220940   
1  34  1001407799     FFF             CF14JA3K30           1004221032   
2  49  1003144658     FFF       MLZ2012N100LT000           1551163438   
3  62  1004220940     FFF             CF14JT120R           1001407707   
4  74  1004221032     FFF             CF14JT3K30           1001407799   

                        mfgPartDesc  \
0              CARBON FILM RESISTOR   
1              CARBON FILM RESISTOR   
2  INDUCTOR FOR DECOUPLING CIRCUITS   
3              CARBON FILM RESISTOR   
4              CARBON FILM RESISTOR   

                                    mfgPkgDesc   mfrName  \
0                                 AXIAL LEADED  SEI Elek   
1                                 AXIAL LEADED  SEI Elek   
2  CHIP, 0805, HALOGEN FREE AND ROHS COMPLIANT       TDK   
3                                 AXIAL LEADED  SEI Elek   
4                                 AXIAL LEADED  SEI Elek   

                                     partDescription partStatus  \
0  Fixed Resistor, Carbon Film, 0.25W, 120ohm, 35...     Active   
1  Fixed Resistor, Carbon Film, 0.25W, 3300ohm, 3...     Active   
2  General Purpose Inductor, 10uH, 20%, 1 Element...     Active   
3  Fixed Resistor, Carbon Film, 0.25W, 120ohm, 35...     Active   
4  Fixed Resistor, Carbon Film, 0.25W, 3300ohm, 3...     Active   

          creationTime  modificationTime  
0  2018-10-09 12:52:32               NaN  
1  2018-10-09 12:52:33               NaN  
2  2018-10-09 12:52:33               NaN  
3  2018-10-09 12:52:33               NaN  
4  2018-10-09 12:52:33               NaN

In [7]:
df_altMatIHS.sort_values(by = "objectId", inplace=True)
df_altMatIHS = df_altMatIHS[["objectId","manufacturerPartNumber","alternativeObjectId"]]

In [8]:
df_master = pd.read_csv(os.path.join(datapath, "masterData_IHS_SE.csv"), sep = "|")
df_master = df_master[["ObjectId",]]

In [10]:
df_master.columns

Index(['ObjectId', 'Description_IHS', 'partType', 'category', 'docURL',
       'DocTitle', 'pubDate', 'commID', 'Description_SE', 'datasheet',
       'SupplierURLDatasheet', 'taxonomyPath', 'Image_small', 'Image_large'],
      dtype='object')

In [8]:
df_cap.dropna(axis = 1, how = "all", inplace = True)
df_cap.shape

/opt/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


(8792, 89)

In [9]:
df_cap.to_csv(os.path.join(datapath, "cap_final.csv"), index = False)

In [5]:
#df_master = store['Final_ABT']
#store.close()
df_master = pd.read_csv(os.path.join(datapath, "ABT_IHS.csv"))
df_master.shape

/opt/anaconda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (4,6,8,13,15,16,19,21,23,29,31,32,37,38,40,42,43,45,46,47,48,52,54,56,59,61,62,69,70,73,77,79,83,84,86,87,88,90,101,103,104,105,106,109,111,112,120,123,125,126,129,133,136,137,138,140,142,143,144,145,146,147,149,150,151,152,153,154,157,159,162,163,166,167,170,172,173,175,177,179,181,182,184,185,187,191,192,193,195,197,200,203,209,210,211,212,213,214,216,220,221,228,234,235,238,239,240,243,252,256,257,259,261,262,264,266,268,272,274,275,284,286,287,290,292,296,297,299,300,301,304,306,309,310,311,313,314,317,321,322,323,326,327,331,332,335,337,338,339,341,345,346,348,351,352,355,359,362,366,368,369,370,371,372,375,377,378,380,381,383,385,386,387,390,392,394,396,399,400,401,404,405,407,414,417,419,420,424,429,430,431,433,435,438,443,444,447,449,454,455,456,457,459,460,468,470,471,472,473,475,477,479,481,484,486,496,499,500,502,503,505,506,508,512,514,515,516,519,520,532,534,545,547,549,5

(58244, 715)

In [8]:
df_SimMap = pd.read_csv(os.path.join(datapath, "SimMap.csv"))
df_SimMap.columns

Index(['PartNumber', 'SimilarPartNumber', 'objectId', 'Description_IHS',
       'partType', 'category', 'Sim_measure'],
      dtype='object')

In [81]:
df_LED = df_SimMap.loc[df_SimMap.category.str.contains("LED") == True]
df_cap = df_SimMap.loc[df_SimMap.category.str.lower().str.contains("capacitor") == True]

In [99]:
df_led = df_LED.merge(df_master,left_on="objectId", right_on="objectId")
#df_cap = df_cap.merge(df_master,left_on="objectId", right_on="objectId")

In [38]:
df_cap.columns.tolist()[50:60]
#"Dielectric".lower() in a.lower()


['Operating_Frequency_Nom__MHz__',
 'Input_Format',
 'RF_Microwave_Device_Type',
 'Board_Mounting_Option',
 'Contact_Finish___Mating',
 'Number_of_Macro_Cells',
 'Dielectric_Strength_Between_Coil_and_Contacts__Vrms__',
 'Primary_Inductance__uH__',
 'Finish',
 'Rated_Current__Signal___A__']

In [20]:
cols_cap = ["Ripple_Current","Number_of_Terminals","Associated_Capacitor_Construction",
            "Capacitance_Value","Capacitor_Type","Construction","Dielectric","Equivalent_Series_Resistance","Equivalent_Series_Resistance_Type","Leakage_Current"
            ,"Lifetime"
            ,"Maximum_Operating_Temperature"
            ,"Minimum_Operating_Temperature"
            ,"Mounting"
            ,"Product_Depth"
            ,"Product_Diameter"
            ,"Product_Height"
            ,"Product_Length"
            ,"Suppression_Class"
            ,"Terminal_Pitch"
            ,"Termination_Style"
            ,"Package_Style"
            ,"Tolerance"
            ,"Type"
            ,"Voltage"]


In [52]:
matching_cols_caps = dict()
for cols in cols_cap:
    matching_cols_caps[cols] = [s for s in df_cap.columns.str.lower() if cols.lower() in s]

In [50]:
cols_led = ["Category","Color","Dominant_Wavelength","Face_Shape","Maximum_Forward_Current",
"Maximum_Forward_Voltage_per_Color",
"Maximum_Supply_Current",
"Minimum_Operating_Temperature",
"Mounting_Orientation",
"Viewing_Angle",
"Maximum_Operating_Temperature",
"Sub_Category"]

matching_cols_leds = dict()
for cols in cols_led:
    matching_cols_leds[cols] = [s for s in df_led.columns.str.lower() if cols.lower() in s]

In [53]:
cols_for_caps = list(itertools.chain.from_iterable(matching_cols_caps.values())) 
cols_for_leds = list(itertools.chain.from_iterable(matching_cols_leds.values()))

In [83]:
df_cap.columns = df_cap.columns.str.upper()
df_cap.columns

Index(['PARTNUMBER', 'SIMILARPARTNUMBER', 'OBJECTID', 'DESCRIPTION_IHS',
       'PARTTYPE', 'CATEGORY', 'SIM_MEASURE', 'OUTER_DIAMETER_MIN__INCH__',
       'ESR__MOHM__', 'CONTACT_VOLTAGE_AC__MAX__V__',
       ...
       'OUTPUT_FUNCTION', 'DROPOUT_VOLTAGE1_NOM__V__',
       'INPUT_VOLTAGE_MIN__V__', 'DMA_CHANNELS', 'PRIMARY_RATING',
       'CONTACT_STYLE', 'CASE_CODE', 'CONTACT_CURRENT_AC__MAX__A__',
       'PEAK_REFLOW_TEMPERATURE__CEL_', 'REFERENCE_VOLTAGE_NOM__V__'],
      dtype='object', length=721)

In [85]:
cols_for_caps = [col.upper() for col in cols_for_caps]
cols_for_caps.append("objectId")
cols_to_drop = [col for col in df_cap.columns if col not in cols_for_caps]

In [128]:
df_cap.drop(columns=cols_to_drop, inplace=True)
df_cap.dropna(axis = 1,how = "all", inplace = True)

KeyError: "['PARTNUMBER' 'SIMILARPARTNUMBER' 'DESCRIPTION_IHS' 'SIM_MEASURE'\n 'OUTER_DIAMETER_MIN__INCH__' 'ESR__MOHM__' 'CONTACT_VOLTAGE_AC__MAX__V__'\n 'DEVICE_SOCKET_TYPE' 'PACKAGE_LENGTH__MM__' 'PARALLEL_SERIAL'\n 'NUMBER_OF_POLES' 'FET_TECHNOLOGY' 'CURRENT_RATING__A__'\n 'CUT_OFF_VOLTAGE__V__' 'CONTACT_BOUNCE_TIME_MAX__MS__'\n 'INPUT_VOLTAGE_ABSOLUTE_MAX__V__' 'OPTOELECTRONIC_DEVICE_TYPE'\n 'COMMON_MODE_REJECT_RATIO_NOM__DB__' 'TURNS_RATIO__NP_NS_' 'OPTION'\n 'SERIES_RESISTANCE__OHM__' 'REACH_COMPLIANT' 'BUS_COMPATIBILITY'\n 'OUTPUT_VOLTAGE_1__V__' 'RELIABILITY' 'BODY_DEPTH__INCH__'\n 'PLATING_THICKNESS__INCH__' 'WIDTH__MM__' 'OUTER_DIAMETER__MM__'\n 'PROPAGATION_DELAY_MAX_NOM_SUP__NS__' 'ANALOG_OUTPUT_VOLTAGE_MIN__V__'\n 'A_D_CONVERTERS' 'RATED_VOLTAGE_AC___V__' 'ACCESS_MODE'\n 'CONTACT_OUTPUT_SUPPLY_TYPE' 'INPUT_VOLTAGE_NOM__V__'\n 'SLEW_RATE_NOM__V_US__' 'COMMON_MODE_REJECT_RATIO_MIN__DB__'\n 'STANDBY_CURRENT_MAX__A__' 'SOLDERABILITY' 'APPROVALS'\n 'ISOLATION_MAX__DB__' 'MATIN

In [74]:
len(cols_to_drop)

555

In [129]:
df_cap.head()

OBJECTID    PARTTYPE  NEGATIVE_TOLERANCE_____ NUMBER_OF_TERMINALS  \
0  1890694877  Capacitors                      NaN                   2   
1  1890694877  Capacitors                      NaN                   2   
2  1890694877  Capacitors                      NaN                   2   
3  1890694877  Capacitors                      NaN                   2   
4  1714676612  Capacitors                      NaN                   2   

   TERMINAL_PITCH__MM__  RIPPLE_CURRENT__MA__ NETWORK_TYPE  \
0                   NaN                   NaN          NaN   
1                   NaN                   NaN          NaN   
2                   NaN                   NaN          NaN   
3                   NaN                   NaN          NaN   
4                   NaN                   NaN          NaN   

   POSITIVE_TOLERANCE_____  LEAKAGE_CURRENT__MA__ DIELECTRIC_MATERIAL  \
0                      NaN                    NaN             CERAMIC   
1                      NaN                    NaN             CERAMIC   
2                      NaN                    NaN             CERAMIC   
3                      NaN                    NaN             CERAMIC   
4                      NaN                    NaN             CERAMIC   

              ...             Output_Function Dropout_Voltage1_Nom__V__  \
0             ...                         NaN                       NaN   
1             ...                         NaN                       NaN   
2             ...                         NaN                       NaN   
3             ...                         NaN                       NaN   
4             ...                         NaN                       NaN   

   Input_Voltage_Min__V__ DMA_Channels  Primary_Rating Contact_Style  \
0                     NaN          NaN             NaN           NaN   
1                     NaN          NaN             NaN           NaN   
2                     NaN          NaN             NaN           NaN   
3                     NaN          NaN             NaN           NaN   
4                     NaN          NaN             NaN           NaN   

   Case_Code Contact_Current_AC__Max__A__ Peak_Reflow_Temperature__Cel_  \
0        NaN                          NaN                           NaN   
1        NaN                          NaN                           NaN   
2        NaN                          NaN                           NaN   
3        NaN                          NaN                           NaN   
4        NaN                          NaN                           NaN   

   Reference_Voltage_Nom__V__  
0                         NaN  
1                         NaN  
2                         NaN  
3                         NaN  
4                         NaN  

[5 rows x 738 columns]

In [ ]:
df_cap = df_cap.merge(df_SimMap, left_on = "OBJECTID", right_on= "objectId")

In [130]:
df_cap.drop_duplicates(inplace=True)

In [132]:
df_cap.columns

Index(['OBJECTID', 'PARTTYPE', 'NEGATIVE_TOLERANCE_____',
       'NUMBER_OF_TERMINALS', 'TERMINAL_PITCH__MM__', 'RIPPLE_CURRENT__MA__',
       'NETWORK_TYPE', 'POSITIVE_TOLERANCE_____', 'LEAKAGE_CURRENT__MA__',
       'DIELECTRIC_MATERIAL',
       ...
       'Output_Function', 'Dropout_Voltage1_Nom__V__',
       'Input_Voltage_Min__V__', 'DMA_Channels', 'Primary_Rating',
       'Contact_Style', 'Case_Code', 'Contact_Current_AC__Max__A__',
       'Peak_Reflow_Temperature__Cel_', 'Reference_Voltage_Nom__V__'],
      dtype='object', length=738)

In [95]:
df_cap.to_csv(os.path.join(datapath, "capacitors.csv"))

### led

In [102]:
cols_for_leds = [col.upper() for col in cols_for_leds]
cols_for_leds.append("objectId")
cols_to_drop = [col for col in df_led.columns if col not in cols_for_leds]

In [100]:
df_led.columns = df_led.columns.str.upper()
df_led.columns
#cols_for_leds

Index(['PARTNUMBER', 'SIMILARPARTNUMBER', 'OBJECTID', 'DESCRIPTION_IHS',
       'PARTTYPE', 'CATEGORY', 'SIM_MEASURE', 'OUTER_DIAMETER_MIN__INCH__',
       'ESR__MOHM__', 'CONTACT_VOLTAGE_AC__MAX__V__',
       ...
       'OUTPUT_FUNCTION', 'DROPOUT_VOLTAGE1_NOM__V__',
       'INPUT_VOLTAGE_MIN__V__', 'DMA_CHANNELS', 'PRIMARY_RATING',
       'CONTACT_STYLE', 'CASE_CODE', 'CONTACT_CURRENT_AC__MAX__A__',
       'PEAK_REFLOW_TEMPERATURE__CEL_', 'REFERENCE_VOLTAGE_NOM__V__'],
      dtype='object', length=721)

In [103]:
cols_to_drop

['PARTNUMBER',
 'SIMILARPARTNUMBER',
 'DESCRIPTION_IHS',
 'PARTTYPE',
 'SIM_MEASURE',
 'OUTER_DIAMETER_MIN__INCH__',
 'ESR__MOHM__',
 'CONTACT_VOLTAGE_AC__MAX__V__',
 'DEVICE_SOCKET_TYPE',
 'PACKAGE_LENGTH__MM__',
 'PARALLEL_SERIAL',
 'NUMBER_OF_POLES',
 'FET_TECHNOLOGY',
 'CURRENT_RATING__A__',
 'CUT_OFF_VOLTAGE__V__',
 'CONTACT_BOUNCE_TIME_MAX__MS__',
 'INPUT_VOLTAGE_ABSOLUTE_MAX__V__',
 'OPTOELECTRONIC_DEVICE_TYPE',
 'COMMON_MODE_REJECT_RATIO_NOM__DB__',
 'TURNS_RATIO__NP_NS_',
 'OPTION',
 'SERIES_RESISTANCE__OHM__',
 'REACH_COMPLIANT',
 'BUS_COMPATIBILITY',
 'OUTPUT_VOLTAGE_1__V__',
 'RELIABILITY',
 'BODY_DEPTH__INCH__',
 'PLATING_THICKNESS__INCH__',
 'WIDTH__MM__',
 'NEGATIVE_TOLERANCE_____',
 'OUTER_DIAMETER__MM__',
 'PROPAGATION_DELAY_MAX_NOM_SUP__NS__',
 'ANALOG_OUTPUT_VOLTAGE_MIN__V__',
 'A_D_CONVERTERS',
 'RATED_VOLTAGE_AC___V__',
 'ACCESS_MODE',
 'CONTACT_OUTPUT_SUPPLY_TYPE',
 'INPUT_VOLTAGE_NOM__V__',
 'SLEW_RATE_NOM__V_US__',
 'COMMON_MODE_REJECT_RATIO_MIN__DB__',
 'STANDB

In [104]:
df_led.drop(columns=cols_to_drop, inplace=True)
df_tmp = df_led.dropna(axis = 1,how = "all")
df_tmp.head()

OBJECTID      CATEGORY SUB_CATEGORY COLOR  VIEWING_ANGLE__DEG__  \
0  2086421300  Visible LEDs          NaN   RED                   NaN   
1  2086421300  Visible LEDs          NaN   RED                   NaN   
2  2084272601  Visible LEDs          NaN   RED                   NaN   
3  2084272601  Visible LEDs          NaN   RED                   NaN   
4  8001912636  Visible LEDs          NaN   NaN                   NaN   

  COLOR_WAVELENGTH  
0              NaN  
1              NaN  
2              NaN  
3              NaN  
4              NaN

In [105]:
df_led = df_tmp.merge(df_SimMap, left_on = "OBJECTID", right_on= "objectId")
df_cap.drop_duplicates(inplace=True)
df_led.to_csv(os.path.join(datapath, "led.csv"))

In [107]:
stop_words = ['nan', 'no', "-999999"]
pattern = "[^\w.+-]"

In [174]:
cols_ = df_cap.columns.tolist()

cols = [col for col in form_cols if col.lower() in df_cap.columns.str.lower()]
cols

['Construction', 'Package_Style']

In [180]:
#df_cap.dropna(axis=1, how="all", inplace=True)
df_cap.columns = df_cap.columns.str.upper()

In [182]:
form_cols = ["Construction".upper(),"BODY_DEPTH__INCH__","INNER_DIAMETER_MIN__INCH__", "DIAMETER__MM__","PACKAGE_DIAMETER__MM__","HEIGHT__MM__","LENGTH__MM__","TERMINATION_TYPE","Package_Style".upper()]
fit_cols = ["MOUNTING_FEATURE","TERMINAL_PITCH__MM__"]
func_cols = [col for col in cols_cap if col not in form_cols + fit_cols]


In [183]:
cleaner_text = PatternCleaner(text_col="text", out_col="text_cleaned", pattern = pattern)
cutter_text = WordCutter(text_col="text_cleaned", out_col="text_cleaned", min_length=1)
stopwords_remover_text = StopwordRemover(text_col="text_cleaned", out_col="text_cleaned", stop_list=stop_words)
pipe_prep_text = Pipeline([("cleaner_text", cleaner_text),
                      ("cutter_text",  cutter_text),
                      ("stopwords_remover_text", stopwords_remover_text)
                     ])

In [184]:
["OBJECTID"] + form_cols

['OBJECTID',
 'CONSTRUCTION',
 'BODY_DEPTH__INCH__',
 'INNER_DIAMETER_MIN__INCH__',
 'DIAMETER__MM__',
 'PACKAGE_DIAMETER__MM__',
 'HEIGHT__MM__',
 'LENGTH__MM__',
 'TERMINATION_TYPE',
 'PACKAGE_STYLE']

In [185]:
#df_cap_fit = df_cap.drop(columns=[col for col in df_cap.columns if col not in ["OBJECTID"] + fit_cols])
df_cap_form = df_cap.drop(columns=[col for col in df_cap.columns if col not in ["OBJECTID"] + form_cols])

In [198]:
df_cap_form.columns
#cols_to_take = df_cap_form.columns.tolist().remove("OBJECTID")
#df_cap_form

Index(['OBJECTID', 'PACKAGE_STYLE', 'OBJECTID', 'DIAMETER__MM__',
       'HEIGHT__MM__', 'PACKAGE_STYLE', 'LENGTH__MM__'],
      dtype='object')

In [202]:
df_cap_form = text_col_builder(df_cap_form, ['PACKAGE_STYLE','DIAMETER__MM__','HEIGHT__MM__', 'PACKAGE_STYLE', 'LENGTH__MM__'])
#cols_to_consider 

/opt/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [203]:
cleaner_text = PatternCleaner(text_col="text", out_col="text_cleaned", pattern = pattern)
cutter_text = WordCutter(text_col="text_cleaned", out_col="text_cleaned", min_length=1)
stopwords_remover_text = StopwordRemover(text_col="text_cleaned", out_col="text_cleaned", stop_list=stop_words)
pipe_prep_text = Pipeline([("cleaner_text", cleaner_text),
                      ("cutter_text",  cutter_text),
                      ("stopwords_remover_text", stopwords_remover_text)
                     ])

Final check if all the column names are unique

In [204]:
data_final = pipe_prep_text.fit_transform(df_cap_form)

/opt/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-

In [205]:
data_final[["text", "text_cleaned"]].head()

text text_cleaned
0                  
1                  
2                  
3                  
4

## Define similarity pipelines for overall, form fit  & function

### Generell similarity

In [206]:
data_final.drop_duplicates(inplace = True)
data_final.shape

/opt/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


(8780, 9)

In [207]:
selector_text = DataFrameSelector("text_cleaned")
tf_idf_text = TfidfVectorizer(stop_words=stop_words,
                         analyzer='char_wb',
                         ngram_range=(1,3),
                         lowercase=True)
pipe_tfidf_text = Pipeline([ ("selector", selector_text),("tf_idf", tf_idf_text)])

In [208]:
X = pipe_tfidf_text.fit_transform(data_final)
print(X.shape)
print(len(data_final))

(8780, 602)
8780


### Similarity based on form fit function

In [155]:
 pd.set_option('max_colwidth', 200)

In [156]:
def cosine_sim_mp(k,
                   X,
                   df,
                  n_neighbors = 10,
                  threshold = 0.8):
    #time.sleep(0.001)
    """
        calculates based on sparse/dense (tf-idf) - matrix the cosine distance between every material. The n_neighbors that are closest
        are then taken from the input dataframe and are saved into a dictionary. The output dictionary will have as key the corresponding 
        index from the dataframe and additionally the n_neighbors from the dataframe. So the values of the dictionary are the 
        dataframes of lenght n_neighbors
        
        Input: 
            k = Iterator
            X = (sparse) matrix
            df = pandas dataframe from which the n_neigbors - closest row items are chosen for index k
            n_neighbors = number of neighbors 
            threshold minimum similarity required to be considered
    """
    
    similarities = dict()
    
    # Check type, is X numpy or sparse matrix?
    if type(X) is np.ndarray:
        tmp = (X[k,] * X.T).flatten()  
    else:
        tmp = (X[k,] * X.T).toarray().flatten() 
    
     
    cos_dist = tmp / np.max(tmp)
    most_sim_n_neighbors = np.argsort(tmp / np.max(tmp))[::-1][:n_neighbors]
    most_sim = [idx for idx in most_sim_n_neighbors if cos_dist[idx] > threshold]
    key_main = df.index[k]    

    #Build the relevant dataframe:
    df_tmp = df.loc[df.index[most_sim]]

    dic_sim = pd.DataFrame({"Sim_measure": cos_dist[most_sim]},
                            index =  df.index[most_sim])

    df_sim_dist = df_tmp.join(dic_sim)
    similarities[key_main] = df_sim_dist.sort_values(by=['Sim_measure'], ascending = False)   
    del tmp, df_sim_dist, cos_dist, dic_sim
    # clean memory : 
    #gc.collect()
    return similarities    

In [209]:
# ### Running the algorithm
# Running the algortihm in parallel so that all cpu are in use. Speeds up calculation ~10 times than singe thread process.
# Caution you may facing memory issues since multiprocessing will take a huge amount of ram by default.
# The most important thing is to set the backend to threading, since loky runs out of memory fast.

# ###  Calculate the Cosine distance between the different materials


N = len(data_final)
# backend = "threading"
results = Parallel(n_jobs=-1, verbose = 1,backend = "threading"  )(delayed(cosine_sim_mp,)(i,X =X , df = data_final,  threshold = 0) for i in range(N))

similarities = { k: v for d in results for k, v in d.items() }
len(similarities.keys())

df_similarityMap = pd.concat(similarities)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
/opt/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in true_divide
/opt/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 1768 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 2418 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 3168 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done 4018 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done 4968 tasks      | elapsed:   19.3s
[Parallel(n_jobs=-1)]: Done 6018 tasks    

In [210]:
df_similarityMap.Sim_measure.describe()

count    70309.000000
mean         0.993528
std          0.022142
min          0.806193
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: Sim_measure, dtype: float64

## Define specific function

In [55]:
def remove_dup_columns(df):
    keep_names = set()
    unique_col_names = df.columns.unique()
    #keep_icols = list()
    for icol, name in enumerate(frame.columns):
         if name not in keep_names:
            keep_names.add(name)
            keep_icols.append(icol)  
    
    return frame.iloc[:, keep_icols]

In [56]:
def delete_duplicate_columns(df):
    """
        The function takes a dataframe object and searchs for duplicates column-names.If the function founds duplicates it performs
        a coalesce function, drops the duplicates and merge the final unique column to the df.
    """

    for col in df.columns.unique():
        
        if df[col].ndim > 1:
            #print(col + " Achtung mehrfach")
            df_tmp = df[col]
            df_tmp.columns = [col + "_" + str(k) for k in range(df[col].ndim)]
            #while k in range(df_se[col].ndim):
            
            
            df_tmp[col + "_unique"] = df_tmp[col + "_" + str(0)]
            for k in range(df[col].ndim - 1):
                df_tmp[col + "_unique"] = df_tmp[col + "_unique"].combine_first(df_tmp[col + "_" + str(k + 1)])

            df_tmp.drop(columns=df_tmp[[col + "_" + str(k) for k in range(df[col].ndim)]], inplace=True)
            #print(df_tmp.head(20))

            df_fin = df.merge(df_tmp, left_index=True, right_index=True)
            df_fin.drop(columns=col, inplace = True)
            df = df_fin    
        
        
    return df

In [57]:
def calc_similarity(df, col1, col2):
    """
        Function that calculates the cosine similarty between 2 columns of a dataframe. 
    """
    sims = list()
    for row in df[[col1, col2]].iterrows():
        r = row[1]
        sims.append(np.min(get_cosine_sim(r.col1, r.col2)))
    
    return sims

def get_cosine_sim(*strs):
    """
        Calculate the similartity of a given number of input strings passed.
        If only emptx strings are passed than nan is returned.
        
        Example: 
            string_1 = "heLLo World"
            string_2 = "Hello world 123"
            get_cosine_sim(string_1, string_2)
    """
    
    if not np.max([word.strip() == "" for word in strs]):
        vectors = [t for t in get_vectors(*strs)]

        return cosine_similarity(vectors)
    else:
        return np.nan
    
def get_vectors(*strs):
    """
        Helper function for performing tf-idf vectorization of input strings
    """
    text = [t for t in strs]
    if np.max([len(word.strip()) for word in text]) > 0:
        vectorizer = TfidfVectorizer(text)
        return vectorizer.fit_transform(text).toarray()
    else:
         pass 

In [58]:
def which_col_to_take(df, col1,col2,datapath = None, filename = None, drop_column = True, verbose = 0 , threshold = 0.95):
    """
        function that decides which column to take if they are identical based on the amount of values that are not Null
        
    """
    N = len(df.loc[(~df[col1].isna()) & (~ df[col2].isna() )])
    check = sum(df[col1] == df[col2]) / N if N else 0
      
    if check > threshold:

        if sum(df[col1].isnull()) < sum(df[col2].isnull()):
            col_to_keep = col1
            if verbose > 0:
                print(col1 + " transfer to final since more non-NULL values")
            if drop_column:
                df.drop(columns = [col2], inplace = True)
            #j += 1

        else:
            #k = j
            col_to_keep = col2
            if verbose > 0:
                print(col2 + " transfer to final since more non-NULL values")
                
            if drop_column:
                df.drop(columns = [col1], inplace = True)
                
        if (datapath or filename):
            with open(filename, 'a') as out:
                split1 = col1.find("_")
                split2 = col2.find("_") 
                if col1[split1:].lower() == col2[split2:].lower(): 
                    var = "Columns " + col1 + " and " + col2 +  " have identical names."
                    out.write(var + '\n')
                
                var = "More than " + str(threshold) + " rows coincide in colums " + col1 + " and " + col2
                out.write(var + '\n')
                var = col_to_keep + " transfer to final since more non-NULL values"
    return df

In [59]:
def pattern_replacer(df, column, pattern_to_replace = None, pattern_to_take = None):
    """
            Function that replaces the string specified in pattern_to_replace (as list) with the correspondent values from 
            pattern_to_take.
        """
    if pattern_to_replace:
        for pattern in zip(pattern_to_replace,pattern_to_take):
            df[column].replace(pattern[0] ,pattern[1], inplace = True) 
            #df[column].replace(pattern[0] ,pattern[1]) 
    return df

In [60]:
def pattern_cleaner(df, column = None):
        """
            Function that eliminates all non character (e.g. \,( etc. ) from the material description and perform individual replacements. 
            Finally all additional whitespaces are deleted.
        """
        # Check whether multiple columns with same names are in df
        if column:
            df[column + "_cleaned"] = (
                    df[column]
                    .apply(lambda x: x if type(x)!=str else re.sub("[^\w.+-]"," " ,x))
                    .apply(lambda x: x if type(x)!=str else re.sub("\s+"," " ,x))
                    .apply(lambda x: x if type(x)!=str else x.lower())
                )
        return df

In [61]:
def extract_numbers(df, column = None):
    """
        Function that converts object column to float therby removing any special characters and letters
    """
    if column:
        #df[column] = [''.join(re.findall("\d+\.?\d+", item)) for item in df[column]]
        df[column] = df[column].str.extract('(\d+)')
        
    return df

In [62]:
def text_col_builder(df, text_cols):
        df['text'] = df[text_cols].apply(lambda x: ' '.join(x.dropna().astype(str)),axis=1)
        return df

## Define specific classes to be used in sklearn pipeline

In [63]:
class ColumnExtractor(BaseEstimator, TransformerMixin):
    """
        Base class. Defines which column to select and in which to store results
        text_col = Input column of dataframe with string to be preprocessed
        out_col = Output column of dataframe where the preprocessed column is stored
    """
    def __init__(self, text_col, out_col):
        self.text_col = text_col
        self.out_col  = out_col
    def fit(self, X, y = None):
        return self
    def transform(self, X):
        return self

class PatternCleaner(ColumnExtractor):
    """
        Cleans input col from specified pattern. Inherits fit from base ColumnExtractor.
        
        Input: 
        text_col = Input column of dataframe with string to be preprocessed
        out_col = Output column of dataframe where the preprocessed column is stored
        pattern = regex with patttern to replace
    """
    def __init__(self, text_col,out_col, pattern = None):       
        super().__init__(text_col, out_col)
        self.pattern = pattern
   
    def transform(self, X):
        X[self.out_col] = (
            X[self.text_col]
            .apply(lambda x: x if type(x)!=str else re.sub(self.pattern, " " ,x))
            .apply(lambda x: x if type(x)!=str else re.sub("\s+"," " ,x))
            .apply(lambda x: x if type(x)!=str else x.lower())
            )
        
        return X

class WordCutter(ColumnExtractor):
    """
        Cleans input col from tokens under a certain length. Inherits fit from base ColumnExtractor.
        
        Input: 
        text_col = Input column of dataframe with string to be preprocessed
        out_col = Output column of dataframe where the preprocessed column is stored
        min_length = minimum length to be kept.
       
    """
    
    def __init__(self, text_col,out_col, min_length):       
        super().__init__(text_col, out_col)
        self.min_length = min_length
   
    def transform(self, X):
        X[self.out_col] = X[self.text_col].apply(lambda x: ' '.join([word for word in x.split() if len(word) >= self.min_length ]))
        return X

class DataFrameSelector(BaseEstimator, TransformerMixin):
    """
        Selects Dataframe column and returns its values as numpy array. Important when scikitlearn algorithms are applied.
        
        Input: 
        attribute_names =  Input column of dataframe with string to be preprocessed       
    """
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
            
    def fit(self, X, y=None):
        return self
    def transform(self,X):
        #print("The folollowing columns are transformed to numpy arrays:" + str(attribute_names) )
        return X[self.attribute_names].values


class StopwordRemover(ColumnExtractor):
    """
        Cleans input col from tokens in a particular stopword list. Inherits fit from base ColumnExtractor.
        
        Input: 
        text_col = Input column of dataframe with string to be preprocessed
        out_col = Output column of dataframe where the preprocessed column is stored
        stop_list = list of stopwords
       
    """
    def __init__(self, text_col,out_col, stop_list):
        super().__init__(text_col, out_col)
        self.stop_list = stop_list   
   
    def transform(self, X):
        X[self.out_col] = X[self.text_col].apply(lambda x: ' '.join([word for word in x.split() if word not in self.stop_list]))
        return X
    
class LemmaTokenizer(object):
    """
        User specific tokenizer to be used in in scikit learn tf-idf or countvectorizer class. Performs Porter-stemming and word
        tokenization from nltk
    """
    
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        return [self.ps.stem(word) for word in word_tokenize(doc)]
    

In [5]:
df_master.replace(to_replace="-999999", value=np.nan, inplace=True)
df_master.head()

objectId                                    Description_IHS  \
partNumber                                                                     
0-0925819-3  8.149227e+09                                Connector Accessory   
0-106506-2   1.425896e+09  D Subminiature Connector, 15 Contact(s), Male,...   
0-1393219-6  1.591385e+09  Power/Signal Relay, SPDT, Momentary, 0.018A (C...   
0-1393236-2  1.624993e+09  Power/Signal Relay, SPDT, Momentary, 0.034A (C...   
0-1393243-4  1.592746e+09  Power/Signal Relay, DPDT, Momentary, 0.033A (C...   

                      partType             category  commID Description_SE  \
partNumber                                                                   
0-0925819-3  Connector Support       Assembly Items     NaN            NaN   
0-106506-2          Connectors    D Type Connectors     NaN            NaN   
0-1393219-6             Relays  Power/Signal Relays     NaN            NaN   
0-1393236-2             Relays  Power/Signal Relays     NaN            NaN   
0-1393243-4             Relays  Power/Signal Relays     NaN            NaN   

            taxonomyPath Outer_Diameter_Min__inch__ ESR__mohm__  \
partNumber                                                        
0-0925819-3          NaN                        NaN         NaN   
0-106506-2           NaN                        NaN         NaN   
0-1393219-6          NaN                        NaN         NaN   
0-1393236-2          NaN                        NaN         NaN   
0-1393243-4          NaN                        NaN         NaN   

            Contact_Voltage_AC__Max__V__            ...              \
partNumber                                          ...               
0-0925819-3                          NaN            ...               
0-106506-2                           NaN            ...               
0-1393219-6                          400            ...               
0-1393236-2                          400            ...               
0-1393243-4                          400            ...               

            Output_Function Dropout_Voltage1_Nom__V__ Input_Voltage_Min__V__  \
partNumber                                                                     
0-0925819-3             NaN                       NaN                    NaN   
0-106506-2              NaN                       NaN                    NaN   
0-1393219-6             NaN                       NaN                    NaN   
0-1393236-2             NaN                       NaN                    NaN   
0-1393243-4             NaN                       NaN                    NaN   

            DMA_Channels Primary_Rating Contact_Style Case_Code  \
partNumber                                                        
0-0925819-3          NaN            NaN           NaN       NaN   
0-106506-2           NaN            NaN           NaN       NaN   
0-1393219-6          NaN            NaN           NaN       NaN   
0-1393236-2          NaN            NaN           NaN       NaN   
0-1393243-4          NaN            NaN           NaN       NaN   

            Contact_Current_AC__Max__A__ Peak_Reflow_Temperature__Cel_  \
partNumber                                                               
0-0925819-3                          NaN                           NaN   
0-106506-2                           NaN                           NaN   
0-1393219-6                            5                           NaN   
0-1393236-2                            6                           NaN   
0-1393243-4                            8                           NaN   

            Reference_Voltage_Nom__V__  
partNumber                              
0-0925819-3                        NaN  
0-106506-2                         NaN  
0-1393219-6                        NaN  
0-1393236-2                        NaN  
0-1393243-4                        NaN  

[5 rows x 721 columns]

In [6]:
cols_to_consider  = [col for col in df_master.columns.tolist() if col != "objectId"]
cols_to_consider

['Description_IHS',
 'partType',
 'category',
 'commID',
 'Description_SE',
 'taxonomyPath',
 'Outer_Diameter_Min__inch__',
 'ESR__mohm__',
 'Contact_Voltage_AC__Max__V__',
 'Device_Socket_Type',
 'Package_Length__mm__',
 'Parallel_Serial',
 'Number_of_Poles',
 'FET_Technology',
 'Current_Rating__A__',
 'Cut_off_Voltage__V__',
 'Contact_Bounce_Time_Max__ms__',
 'Input_Voltage_Absolute_Max__V__',
 'Optoelectronic_Device_Type',
 'Common_mode_Reject_Ratio_Nom__dB__',
 'Turns_Ratio__Np_Ns_',
 'Option',
 'Series_Resistance__ohm__',
 'REACH_Compliant',
 'Bus_Compatibility',
 'Output_Voltage_1__V__',
 'Reliability',
 'Body_Depth__inch__',
 'Plating_Thickness__inch__',
 'Width__mm__',
 'Negative_Tolerance_____',
 'Outer_Diameter__mm__',
 'Propagation_Delay_Max_Nom_Sup__ns__',
 'Analog_Output_Voltage_Min__V__',
 'A_D_Converters',
 'Rated_Voltage_AC___V__',
 'Access_Mode',
 'Contact_Output_Supply_Type',
 'Input_Voltage_Nom__V__',
 'Slew_Rate_Nom__V_us__',
 'Common_mode_Reject_Ratio_Min__dB__',
 

In [7]:
pattern = "[^\w.+-]"

In [8]:
stop_words = ['nan', 'no', "-999999"]

In [11]:
df_SimMap = text_col_builder(df_master, cols_to_consider)
#cols_to_consider 
df_SimMap.text.head(10)

partNumber
0-0925819-3    Connector Accessory Connector Support Assembly...
0-106506-2     D Subminiature Connector, 15 Contact(s), Male,...
0-1393219-6    Power/Signal Relay, SPDT, Momentary, 0.018A (C...
0-1393236-2    Power/Signal Relay, SPDT, Momentary, 0.034A (C...
0-1393243-4    Power/Signal Relay, DPDT, Momentary, 0.033A (C...
0-1393260-7    Safety Relay, 6PST, Momentary, 0.05A (Coil), 2...
0-1393644-9    Board Euro Connector, 48 Contact(s), 2 Row(s),...
0-1393763-1    Dry Reed Relay, 1 Form A, SPST, Momentary, 0.0...
0-1393774-1    RF Relay, SPDT, Momentary, 0.014A (Coil), 5VDC...
0-1393774-2    RF Relay, SPDT, Momentary, 0.005A (Coil), 12VD...
Name: text, dtype: object

In [13]:
cleaner_text = PatternCleaner(text_col="text", out_col="text_cleaned", pattern = pattern)
cutter_text = WordCutter(text_col="text_cleaned", out_col="text_cleaned", min_length=2)
stopwords_remover_text = StopwordRemover(text_col="text_cleaned", out_col="text_cleaned", stop_list=stop_words)
pipe_prep_text = Pipeline([("cleaner_text", cleaner_text),
                      ("cutter_text",  cutter_text),
                      ("stopwords_remover_text", stopwords_remover_text)
                     ])

Final check if all the column names are unique

In [14]:
data_final = pipe_prep_text.fit_transform(df_SimMap)

In [15]:
data_final[["text", "text_cleaned"]].head()

text  \
partNumber                                                       
0-0925819-3  Connector Accessory Connector Support Assembly...   
0-106506-2   D Subminiature Connector, 15 Contact(s), Male,...   
0-1393219-6  Power/Signal Relay, SPDT, Momentary, 0.018A (C...   
0-1393236-2  Power/Signal Relay, SPDT, Momentary, 0.034A (C...   
0-1393243-4  Power/Signal Relay, DPDT, Momentary, 0.033A (C...   

                                                  text_cleaned  
partNumber                                                      
0-0925819-3  connector accessory connector support assembly...  
0-106506-2   subminiature connector 15 contact male solder ...  
0-1393219-6  power signal relay spdt momentary 0.018a coil ...  
0-1393236-2  power signal relay spdt momentary 0.034a coil ...  
0-1393243-4  power signal relay dpdt momentary 0.033a coil ...

## Define similarity pipelines for overall, form fit  & function

### Generell similarity

In [16]:
selector_text = DataFrameSelector("text_cleaned")
tf_idf_text = TfidfVectorizer(min_df = 2,
                         max_df=0.9,
                         stop_words=stop_words,
                         tokenizer=LemmaTokenizer(),
                         lowercase=True)
pipe_tfidf_text = Pipeline([ ("selector", selector_text),("tf_idf", tf_idf_text)])

In [17]:
X = pipe_tfidf_text.fit_transform(data_final)
print(X.shape)
print(len(data_final))

(58244, 14542)
58244


### Similarity based on form fit function

In [33]:
df_master.loc[df_master.index == "0-0925819-3"]

objectId      Description_IHS           partType  \
partNumber                                                          
0-0925819-3  8.149227e+09  Connector Accessory  Connector Support   

                   category  commID Description_SE taxonomyPath  \
partNumber                                                        
0-0925819-3  Assembly Items     NaN            NaN          NaN   

            Outer_Diameter_Min__inch__ ESR__mohm__  \
partNumber                                           
0-0925819-3                        NaN         NaN   

            Contact_Voltage_AC__Max__V__  \
partNumber                                 
0-0925819-3                          NaN   

                                                              ...                                                     \
partNumber                                                    ...                                                      
0-0925819-3                                                   ...                                                      

            Input_Voltage_Min__V__ DMA_Channels Primary_Rating Contact_Style  \
partNumber                                                                     
0-0925819-3                    NaN          NaN            NaN           NaN   

            Case_Code Contact_Current_AC__Max__A__  \
partNumber                                           
0-0925819-3       NaN                          NaN   

            Peak_Reflow_Temperature__Cel_ Reference_Voltage_Nom__V__  \
partNumber                                                             
0-0925819-3                           NaN                        NaN   

                                                                                                                text  \
partNumber                                                                                                             
0-0925819-3  Connector Accessory Connector Support Assembly Items 337526 Yes Low Med 0925819 Med CONNECTOR ACCESSORY   

                                                                                                        text_cleaned  
partNumber                                                                                                            
0-0925819-3  connector accessory connector support assembly items 337526 yes low med 0925819 med connector accessory  

[1 rows x 723 columns]

In [31]:
data_final["text_cleaned"].loc[data_final.index[most_sim]].head()

partNumber
0-0925819-3    connector accessory connector support assembly items 337526 yes low med 0925819 med connector accessory
3030491            connector accessory connector support assembly items 148528 low med 3030491 med connector accessory
3030844            connector accessory connector support assembly items 148533 low med 3030844 med connector accessory
3248033            connector accessory connector support assembly items 148598 low med 3248033 med connector accessory
3213976            connector accessory connector support assembly items 148596 low med 3213976 med connector accessory
Name: text_cleaned, dtype: object

In [30]:
 pd.set_option('max_colwidth', 200)

In [34]:
def cosine_sim_mp(k,
                   X,
                   df,
                  n_neighbors = 10,
                  threshold = 0.8):
    #time.sleep(0.001)
    """
        calculates based on sparse/dense (tf-idf) - matrix the cosine distance between every material. The n_neighbors that are closest
        are then taken from the input dataframe and are saved into a dictionary. The output dictionary will have as key the corresponding 
        index from the dataframe and additionally the n_neighbors from the dataframe. So the values of the dictionary are the 
        dataframes of lenght n_neighbors
        
        Input: 
            k = Iterator
            X = (sparse) matrix
            df = pandas dataframe from which the n_neigbors - closest row items are chosen for index k
            n_neighbors = number of neighbors 
            threshold minimum similarity required to be considered
    """
    
    similarities = dict()
    
    # Check type, is X numpy or sparse matrix?
    if type(X) is np.ndarray:
        tmp = (X[k,] * X.T).flatten()  
    else:
        tmp = (X[k,] * X.T).toarray().flatten() 
    
     
    cos_dist = tmp / np.max(tmp)
    most_sim_n_neighbors = np.argsort(tmp / np.max(tmp))[::-1][:n_neighbors]
    most_sim = [idx for idx in most_sim_n_neighbors if cos_dist[idx] > threshold]
    key_main = df.index[k]    

    #Build the relevant dataframe:
    df_tmp = df.loc[df.index[most_sim]]

    dic_sim = pd.DataFrame({"Sim_measure": cos_dist[most_sim]},
                            index =  df.index[most_sim])

    df_sim_dist = df_tmp.join(dic_sim)
    similarities[key_main] = df_sim_dist.sort_values(by=['Sim_measure'], ascending = False)   
    del tmp, df_sim_dist, cos_dist, dic_sim
    # clean memory : 
    #gc.collect()
    return similarities    

In [35]:
# ### Running the algorithm
# Running the algortihm in parallel so that all cpu are in use. Speeds up calculation ~10 times than singe thread process.
# Caution you may facing memory issues since multiprocessing will take a huge amount of ram by default.
# The most important thing is to set the backend to threading, since loky runs out of memory fast.

# ###  Calculate the Cosine distance between the different materials


N = len(data_final)
# backend = "threading"
results = Parallel(n_jobs=-1, verbose = 1,backend = "threading"  )(delayed(cosine_sim_mp,)(i,X =X , df = data_final) for i in range(N))

similarities = { k: v for d in results for k, v in d.items() }
len(similarities.keys())

df_similarityMap = pd.concat(similarities)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 1768 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 2418 tasks      | elapsed:   18.5s
[Parallel(n_jobs=-1)]: Done 3168 tasks      | elapsed:   24.3s
[Parallel(n_jobs=-1)]: Done 4018 tasks      | elapsed:   30.8s
[Parallel(n_jobs=-1)]: Done 4968 tasks      | elapsed:   38.5s
[Parallel(n_jobs=-1)]: Done 6018 tasks      | elapsed:   47.0s
[Parallel(n_jobs=-1)]: Done 7168 tasks      | elapsed:   56.5s
[Parallel(n_jobs=-1)]: Done 8418 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 9768 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 11218 tasks 

### Define how to deal with speficic patterns

In [45]:
df_similarityMap = pd.concat(similarities)

In [ ]:
df_similarityMap["PartNumber"] = df_similarityMap.index.get_level_values(0)
df_similarityMap["SimilarPartNumber"] = df_similarityMap.index.get_level_values(1)
df_similarityMap.reset_index(inplace = True)

In [53]:
df_output =  df_similarityMap[["PartNumber", "SimilarPartNumber","objectId","Description_IHS", "partType", "category", "Sim_measure"]]

In [55]:
df_output.objectId = df_output.objectId.astype("int64")
df_output.loc[df_output.Sim_measure < .9].head()

/usr/local/lib/python3.5/dist-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


PartNumber  SimilarPartNumber    objectId  \
11  0-106506-2            1688793  1827700114   
12  0-106506-2  V23529-B1223-C215  2027283872   
13  0-106506-2         3-338169-2  1719485984   
14  0-106506-2     D09P13A6RV12LF  1786887065   
15  0-106506-2        09653613711  1648132229   

                                                         Description_IHS  \
11         D Subminiature Connector, 9 Contact(s), Male, Solder Terminal   
12        D Subminiature Connector, 15 Contact(s), Male, Solder Terminal   
13  D Subminiature Connector, 15 Contact(s), Male, Solder Terminal, Plug   
14         D Subminiature Connector, 9 Contact(s), Male, Solder Terminal   
15        D Subminiature Connector, 25 Contact(s), Male, Solder Terminal   

      partType           category  Sim_measure  
11  Connectors  D Type Connectors     0.850741  
12  Connectors  D Type Connectors     0.834972  
13  Connectors  D Type Connectors     0.817167  
14  Connectors  D Type Connectors     0.806422  
15  Connectors  D Type Connectors     0.804815

In [59]:
#store.close()
#df_output.to_hdf(store, key = "SimMap" , mode = "w")
df_output.to_csv(os.path.join(datapath, "SimMap.csv"), index = False)
df_master.to_csv(os.path.join(datapath, "ABT_IHS.csv"), index = False)

In [57]:
store.open()
df_test = store["SimMap"]
df_test.head()

PartNumber SimilarPartNumber    objectId      Description_IHS  \
0  0-0925819-3       0-0925819-3  8149227249  Connector Accessory   
1  0-0925819-3           3030491  1765017628  Connector Accessory   
2  0-0925819-3           3030844  1765017632  Connector Accessory   
3  0-0925819-3           3248033  1765017813  Connector Accessory   
4  0-0925819-3           3213976  1765017797  Connector Accessory   

            partType        category  Sim_measure  
0  Connector Support  Assembly Items          1.0  
1  Connector Support  Assembly Items          1.0  
2  Connector Support  Assembly Items          1.0  
3  Connector Support  Assembly Items          1.0  
4  Connector Support  Assembly Items          1.0

In [31]:
cols_to_consider = [cols for cols in cols_to_consider if cols not in ("Description_IHS", "partType", "category", "text_cleaned","Sim_measure")]
df_similarityMap.drop(columns= cols_to_consider, inplace=True)
#len(df_similarityMap.columns)
#len(data_final.columns)
#len(cols_to_consider)

In [33]:
df_similarityMap.head()

objectId  \
            partNumber                  
0-0925819-3 0-106506-2   1.425896e+09   
            0-1393644-9  2.130192e+09   
            0-1393236-2  1.624993e+09   
            0-1393219-6  1.591385e+09   
            0-1393243-4  1.592746e+09   

                                                           Description_IHS  \
            partNumber                                                       
0-0925819-3 0-106506-2   D Subminiature Connector, 15 Contact(s), Male,...   
            0-1393644-9  Board Euro Connector, 48 Contact(s), 2 Row(s),...   
            0-1393236-2  Power/Signal Relay, SPDT, Momentary, 0.034A (C...   
            0-1393219-6  Power/Signal Relay, SPDT, Momentary, 0.018A (C...   
            0-1393243-4  Power/Signal Relay, DPDT, Momentary, 0.033A (C...   

                           partType                          category  \
            partNumber                                                  
0-0925819-3 0-106506-2   Connectors                 D Type Connectors   
            0-1393644-9  Connectors  Headers and Edge Type Connectors   
            0-1393236-2      Relays               Power/Signal Relays   
            0-1393219-6      Relays               Power/Signal Relays   
            0-1393243-4      Relays               Power/Signal Relays   

                                                                      text  \
            partNumber                                                       
0-0925819-3 0-106506-2   D Subminiature Connector, 15 Contact(s), Male,...   
            0-1393644-9  Board Euro Connector, 48 Contact(s), 2 Row(s),...   
            0-1393236-2  Power/Signal Relay, SPDT, Momentary, 0.034A (C...   
            0-1393219-6  Power/Signal Relay, SPDT, Momentary, 0.018A (C...   
            0-1393243-4  Power/Signal Relay, DPDT, Momentary, 0.033A (C...   

                                                              text_cleaned  \
            partNumber                                                       
0-0925819-3 0-106506-2   subminiature connector 15 contact male solder ...   
            0-1393644-9  board euro connector 48 contact row male solde...   
            0-1393236-2  power signal relay spdt momentary 0.034a coil ...   
            0-1393219-6  power signal relay spdt momentary 0.018a coil ...   
            0-1393243-4  power signal relay dpdt momentary 0.033a coil ...   

                         Sim_measure  
            partNumber                
0-0925819-3 0-106506-2      0.264851  
            0-1393644-9     0.198421  
            0-1393236-2     0.008025  
            0-1393219-6     0.007877  
            0-1393243-4     0.007731

In [91]:
import sys
print("Number of rows of data: " , len(df_similarityMap))
print("Storage in GB used: ", sys.getsizeof(df_similarityMap) / 1e9)

Number of rows of data:  492536
Storage in GB used:  2.67854346


In [93]:
print(datapath)

C:\Users\z000sypa\Projects\PLM\data


In [95]:
df_reduced = df_similarityMap.head(10000)
#print("Storage in GB used: ", sys.getsizeof(df_reduced) / 1e9)
df_reduced.to_csv(os.path.join(datapath, "SimilarityResistors_First10000.csv"), encoding = "utf-8")

## Store into the database

In [ ]:
con_to_store = pass

df_similarityMap.to_sql(con=con_to_store, name='SimilarityMap_resistors', if_exists='replace', flavor='mysql')